# TP0: Introducción al Análisis de Datos

### Imports y útiles, ejecutar primero

In [241]:
from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect
import json
import pandas as pd
import seaborn as sns
from enum import Enum

factory = PokemonFactory("pokemon.json")
class Pokeballs(Enum):
    POKEBALL = "pokeball"
    ULTRABALL = "ultraball"
    FASTBALL = "fastball"
    HEAVYBALL = "heavyball"

### 1A: Probabilidad de captura promedio para cada pokemon con cada pokebola

Se toman como condiciones "ideales", nivel 100, sin estado (esto no lo dice el enunciado) y con 100% de HP. No obstante, sabiendo ya la respuesta quisiera hacer una aclaración. 
Cuando se nombran las condiciones "ideales" en el primer punto, deberían ser estas HP:1% (o el mínimo numero entero si no es en porcentaje, HP:1) y LVL 100, ya que el entrenador quiere atrapar el pokemon más fuerte (aunque no se encuentren niveles 100 salvajes) con la mayor probabilidad de captura posible (y esto se obtiene cuando el pokemon tiene la menor cantidad de vida posible y un estado favorable como SLEEP o FREEZE).

Retomando, teniendo en cuenta cada Pokemon dentro de la base de datos (pokemon.json) y ejecutando 100 veces, cada vez, la función de catching con cada pokebola disponible, obtenemos una tabla que resume los porcentajes de captura en cada una de estas situaciones.

In [243]:
with open("pokemon.json", "r") as c:
  pokemon_db = json.load(c)
  probabilities = {}
  for pokemon in pokemon_db:
    probabilities[pokemon] = {}
    pk = factory.create(pokemon, 100, StatusEffect.NONE, 1)
    for pokeball in Pokeballs:
      captured = 0
      for _ in range(100):
        captured += 1 if attempt_catch(pk, pokeball.value)[0] else 0
      probabilities[pokemon][pokeball.value] = captured

df = pd.DataFrame(probabilities)
df.style.background_gradient(cmap="Greens")
			

,jolteon,caterpie,snorlax,onix,mewtwo
pokeball,5,30,5,6,2
ultraball,20,72,9,6,0
fastball,19,24,7,6,3
heavyball,2,28,10,9,0


### 1B: Efectividad en las pokebolas

Por lo visto anteriormente, donde se evalúa el porcentaje de captura, vemos que para distintos pokemon se obtienen diferentes porcentajes de captura. Esto indica que hay propiedades intrínsecas en los pokemon que hace que el porcentaje de captura sea distinto (esto es así ya que es lo único que está cambiando, pues la pokebola, el nivel, el porcentaje de HP y el estado del pokemon sigue siendo el mismo).

Se cambia el eje del gradiente para una mejor visualización de izquierda a derecha.

In [244]:
df.style.background_gradient(cmap="Greens", axis=1)

,jolteon,caterpie,snorlax,onix,mewtwo
pokeball,5,30,5,6,2
ultraball,20,72,9,6,0
fastball,19,24,7,6,3
heavyball,2,28,10,9,0


### 2A: Condiciones de salud

Tomamos todos los pokemon en todas las condiciones de salud posibles, y los intentamos capturar 100 veces en cada situación. El resultado muestra que existe una variación según el estado en el que se encuentra el pokemon. Se aprecia que sleep y freeze son los más fuertes pero también que poison, burn y paralysis aumentan el porcentaje.

In [249]:
with open("pokemon.json", "r") as c:
  pokemon_db = json.load(c)
  probabilities = {}
  for pokemon in pokemon_db:
    probabilities[pokemon] = {}
    for status in StatusEffect:
      pk = factory.create(pokemon, 100, status, 1)
      captured = 0
      for _ in range(100):
        captured += 1 if attempt_catch(pk, "pokeball")[0] else 0
      probabilities[pokemon][status.value[0]] = captured

df = pd.DataFrame(probabilities)
df.style.background_gradient(cmap="Greens")

,jolteon,caterpie,snorlax,onix,mewtwo
poison,7,56,4,7,0
burn,13,48,5,12,0
paralysis,5,44,5,11,1
sleep,8,55,13,17,0
freeze,15,64,10,14,0
none,4,33,6,9,0


### 2B: Puntos de vida

Para demostrar que los puntos de vida tienen importancia a la hora de capturar un pokemon, tomamos como muestras todos los pokemon disponibles (para demostrar que no sólo uno tiene esta propiedad) con diferentes porcentajes de salud: 100, 75, 50, 25, 1. Los sometimos a capturas con una misma pokebola (la pokeball, para que no tenga ningún modificador extra) y obtuvimos diferentes porcentajes que son inversamente proporcionales al porcentaje de salud del pokemon.

In [250]:
with open("pokemon.json", "r") as c:
  pokemon_db = json.load(c)
  probabilities = {}
  for pokemon in pokemon_db:
    probabilities[pokemon] = {}
    for hp in [1, 0.75, 0.5, 0.25, 0.01]:
      pk = factory.create(pokemon, 100, StatusEffect.NONE, hp)
      captured = 0
      for _ in range(100):
        captured += 1 if attempt_catch(pk, "pokeball")[0] else 0
      probabilities[pokemon][hp] = captured

df = pd.DataFrame(probabilities)
df.style.background_gradient(cmap="Greens")



,jolteon,caterpie,snorlax,onix,mewtwo
1.000000,8,41,1,2,0
0.750000,10,48,5,10,0
0.500000,20,57,8,11,1
0.250000,17,77,7,17,2
0.010000,25,100,11,15,1


### 2C: Parámetros que más afectan la efectividad de captura

La pokebola usada es uno de los parámetros que más afecta la efectividad, ya que cambia mucho usar una heavyball en un Snorlax que no usarla, o usar una fastball en un Jolteon. También vimos que la ultraball aumenta el porcentaje de captura en gran medida ya que vemos hasta un x4 en las muestras obtenidas, teniendo como referencia la pokeball.
Otro parámetro es la vida actual, donde vemos un x2 (comparando desde 100% a 1% de HP) en pokemon fáciles de capturar como Caterpie, o un x10 en pokemon más difíciles como Snorlax u Onix.
Vale aclarar que en todos estos casos estamos aplicando lo que se llama en Economía el supuesto Ceteris Paribus, es decir, la suposición de que todas las demás variables que podrían afectar el resultado en cuestión se mantienen constantes.